In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import numpy as np
import pickle

In [2]:
file_path = 'data\\Churn_Modelling.csv'

df = pd.read_csv(file_path)
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


#### Dropping irrelevant columns

In [3]:
col = ['RowNumber', 'CustomerId', 'Surname']
df.drop(columns = col , axis = 1 , inplace = True)

In [4]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


#### Label Encoding Geography and Gender

In [5]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])


In [6]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0


In [7]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

#### One Hot Encode Geography

In [8]:
ohe_geo = OneHotEncoder()

geo_encoder = ohe_geo.fit_transform(df[['Geography']])


In [9]:
cols = ohe_geo.get_feature_names_out(['Geography'])

In [10]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray().astype(int), columns=cols)

In [11]:
geo_encoded_df.head(2)

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0
1,0,0,1


##### Combine the Ohe columns with the original data

In [12]:
df = pd.concat([df.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

In [13]:
df.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1


#### Save the label encoder and ohe encoder for later

In [14]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('ohe_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

In [15]:
df.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1


#### Divide Dependent and Independent Features

In [16]:
X = df.drop('Exited', axis = 1)
y = df['Exited']

In [17]:
# Dataset Splitting
X_train, X_test , y_train , y_test = train_test_split(X,y , test_size = 0.2 , random_state = 42)

#Scalling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [18]:
#Saving this scaler into pickle file
with open('sc.pkl', 'wb') as file:
    pickle.dump(sc , file)

### ANN Implementations

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [20]:
(X_train.shape[1],)

(12,)

#### Build our ANN Model

In [23]:
model = Sequential([
    #First Hidden Layer Connected to the Input Layer
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

d:\Deep_Learning_Projects\Ann_Classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

#### Compiling the model

In [46]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.keras.losses.BinaryCrossentropy()

In [47]:
model.compile(optimizer=opt , loss = 'binary_crossentropy', metrics=['accuracy'])

### Set up the TensorBoard

In [48]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = 'logs/fit' + datetime.datetime.now().strftime('%Y%m$d_%H%M%S')
tensorflow_callback = TensorBoard(log_dir = log_dir , histogram_freq=1)

### Set up Early Stopping

In [49]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

# Training the Model

In [50]:
history = model.fit(
    X_train, y_train,
    validation_data= (X_test , y_test),
    epochs = 100 , 
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8763 - loss: 0.2921 - val_accuracy: 0.8590 - val_loss: 0.3407
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8817 - loss: 0.2877 - val_accuracy: 0.8570 - val_loss: 0.3433
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8888 - loss: 0.2731 - val_accuracy: 0.8565 - val_loss: 0.3443
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8861 - loss: 0.2777 - val_accuracy: 0.8560 - val_loss: 0.3458
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8885 - loss: 0.2791 - val_accuracy: 0.8565 - val_loss: 0.3474
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8826 - loss: 0.2773 - val_accuracy: 0.8580 - val_loss: 0.3523
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8837 - loss: 0.2758 - val_accuracy: 0.8570 - val_loss: 0.3523
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8889 - loss: 0.2737 - val_accu

### Saving the model

In [51]:
model.save('model.h5')

#### Load Tensorboard Extension

In [37]:
%load_ext tensorboard

In [41]:
# %tensorboard --logdir logs/fit